In [1]:
import os
import cv2
from tensorflow import keras
import numpy as np

In [51]:
file='dataset'
path=os.listdir(file)
categories=[i for i in range(len(path))]
dictonary={i:j for i,j in enumerate(path)}
dictonary

{0: 'with mask', 1: 'without mask'}

In [3]:
data=[]
labels=[]
for category in categories:
    file_path=os.path.join(file,dictonary[category])
    image_path=os.listdir(file_path)
    for i,j in enumerate(image_path):
        images=os.path.join(file_path,image_path[i])
        grey_img=cv2.cvtColor(cv2.imread(images),cv2.COLOR_BGR2GRAY)
        resized=cv2.resize(grey_img,(100,100))
        data.append(resized)
        labels.append(category)

In [4]:

data=np.array(data,'float32')/255
data=np.array(data,'float32').reshape(data.shape[0],100,100,1)
labels=keras.utils.to_categorical(labels,2)


In [5]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Flatten,Dropout,MaxPooling2D,Convolution2D,Dense

In [15]:
model=Sequential()
model.add(Convolution2D(200,3,activation='relu',input_shape=data.shape[1:]))
model.add(MaxPooling2D((2,2),2))

model.add(Convolution2D(100,3,activation='relu',input_shape=data.shape[1:]))
model.add(MaxPooling2D((2,2),2))

model.add(Flatten())

model.add(Dense(50,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2,activation='softmax'))
          
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
train_x,train_y,test_x,test_y=train_test_split(data,labels,test_size=0.1,random_state=4)

In [19]:
hiss=model.fit(train_x,test_x,epochs=20,validation_data=(train_y,test_y),validation_split=)

Epoch 1/20
35/35 [==============================] - 64s 2s/step - loss: 0.7302 - accuracy: 0.5173 - val_loss: 0.6467 - val_accuracy: 0.6123
Epoch 2/20
35/35 [==============================] - 63s 2s/step - loss: 0.5753 - accuracy: 0.7173 - val_loss: 0.4403 - val_accuracy: 0.8225
Epoch 3/20
35/35 [==============================] - 64s 2s/step - loss: 0.3715 - accuracy: 0.8518 - val_loss: 0.3521 - val_accuracy: 0.8478
Epoch 4/20
35/35 [==============================] - 66s 2s/step - loss: 0.2838 - accuracy: 0.8818 - val_loss: 0.3472 - val_accuracy: 0.8261
Epoch 5/20
35/35 [==============================] - 64s 2s/step - loss: 0.2294 - accuracy: 0.9118 - val_loss: 0.2528 - val_accuracy: 0.9022
Epoch 6/20
35/35 [==============================] - 64s 2s/step - loss: 0.1661 - accuracy: 0.9373 - val_loss: 0.2640 - val_accuracy: 0.8986
Epoch 7/20
35/35 [==============================] - 64s 2s/step - loss: 0.1320 - accuracy: 0.9518 - val_loss: 0.1772 - val_accuracy: 0.9167
Epoch 8/20
35/35 [==

In [39]:
fin_json=model.to_json()
with open ('fin.json','w') as json_file:
    json_file.write(fin_json)
model.save_weights('fe.h5')

In [47]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image as img

In [42]:
model=model_from_json(open('fin.json','r').read())

In [38]:
model.evaluate(train_y,test_y)

9/9 [==============================] - 4s 445ms/step - loss: 0.2390 - accuracy: 0.9565


[0.2389936000108719, 0.95652174949646]

In [43]:
model.load_weights('fe.h5')

In [57]:
face_cls=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
ma={0:'Mask',1:'No Mask'}
color={0:(0,255,0),1:(0,0,255)}
color[0]

(0, 255, 0)

In [ ]:
while True:
    ret,frame=cap.read()
    if not ret:
        continue
    grey_img=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face=face_cls.detectMultiScale(grey_img,1.3,5)
    for (x,y,w,h) in face:
        cropped_img=frame[x:x+w,y:y+h]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        grey_cropped=cv2.cvtColor(cropped_img,cv2.COLOR_BGR2GRAY)
        resized=cv2.resize(grey_cropped,(100,100))
        samples=img.img_to_array(resized)
        samples=np.array(samples/255.0,'float32').reshape(1,100,100,1)
        prediction=model.predict(samples)
        index=np.argmax(prediction[0])
        cv2.putText(frame,ma[index],(x,y-9),cv2.FONT_ITALIC,1,color[index],2)
        cv2.imshow('face',frame)
    if cv2.waitKey(10)==13:
        break
cap.release()
cv2.destroyAllWindows()